In [1]:
import pandas as pd
import numpy as np

In [ ]:
df_raw: pd.DataFrame = pd.read_csv(
    'data/processed_data.csv',
    parse_dates=["Time"],
    index_col=[0],
    sep=';'
)

threshold = 1500
df = df_raw[df_raw['Corrente motor'] > threshold]

# Drop all rows with NaN values
df = df.dropna().copy()

In [ ]:
def create_vibration_windows(df, feature_column='feature', window_size=30, threshold=1.5):
    # Step 1: Identify indices where the feature column value is greater than the threshold
    end_indices = np.where(df[feature_column] > threshold)[0]

    # Step 2: Create windows of the specified size ending at these indices
    windows = []
    used_indices = set()

    for idx in end_indices:
        start_idx = idx - (window_size - 1)  # Calculate the start of the window
        if start_idx < 0:
            continue  # Skip windows that start before the beginning of the DataFrame

        # Check if the window overlaps with any already used indices
        if any(i in used_indices for i in range(start_idx, idx + 1)):
            continue  # Skip overlapping windows

        # Add the window to the list of windows
        windows.append(df.iloc[start_idx:idx + 1])

        # Mark the indices as used
        used_indices.update(range(start_idx, idx + 1))

    return windows

def create_non_vibration_windows(df, critical_windows, feature_column='feature', threshold=1.5):
    # Criar janelas não críticas (exemplo: janelas aleatórias abaixo do threshold)
    non_critical_windows = []
    for _ in range(len(critical_windows)):  # Criar o mesmo número de janelas não críticas
        start_idx = np.random.randint(0, len(df) - 30)  # Escolher um índice aleatório
        window = df.iloc[start_idx:start_idx + 30]
        if window[feature_column].max() <= threshold:  # Garantir que a janela não é crítica
            non_critical_windows.append(window)

    return non_critical_windows

In [ ]:
critical_windows = create_vibration_windows(df)
non_critical_windows = create_non_vibration_windows(df, critical_windows)

# Rotular as janelas
critical_labels = [1] * len(critical_windows)  # 1 para críticas
non_critical_labels = [0] * len(non_critical_windows)  # 0 para não críticas

# Combinar janelas e rótulos
windows = critical_windows + non_critical_windows
labels = critical_labels + non_critical_labels

# Converter janelas para numpy array (formato adequado para modelos)
X = np.array([window.values for window in windows])  # Usar todas as colunas da janela
y = np.array(labels)

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Construir o modelo
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),  # (timesteps, n_features)
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Saída binária (crítica ou não crítica)
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Acurácia: {accuracy:.2f}")